# Dimensionality Reduction

Many machine learning problems involve thousands or even millions of features for each training instance. Not only does this make training extremely slow, it can also make it much harder to find a good solution. This problem is often referred to as the *curse of dimensionality*.

Fortunately, in real-world problems, it is often possible to reduce the number of features considerably, turning an intractable problem into a tractable one. For example, consider the MNIST images: the pixels on the image borders are almost always white, so you could completely drop these pixels from the training set without losing much information. Moreover, two neighboring pixels are often highly correlated: if you merge them into a single pixel (e.g., by taking the mean of the two pixel intensities), you will not lose much information.

Apart from speeding up training, dimensionality reduction is also extremely useful for data visualisation. Reducing the number of dimensions down to two (or three) makes it possible to plot a condensed view of a high-dimensional training set on a graph & often gain some important insights by visually detecting patterns, such as clusters. Moreover, data visualisation is essential to communicate your conclusions to people who are not data scientists, in particular, decison makers who will use your results.

---

# The Curse of Dimensionality

We are so used to living in three dimensions that our intuition fails us when we try to imagine a high-dimensional space. Even a basic 4D hypercube is incredibly hard to picture in our mind, let alone a 200-dimensional ellipsoid bent in a 1,000-dimensional space.

<img src = "Images/Multi-Dimensions.png" width = "500" style = "margin:auto"/>

It turns out that many things behave very differently in a high-dimensional space. For example, if you pick a random point in a unit square (1 x 1 square), it will have a 0.4% chance of being located less than 0.001 from a border (in other words, it will be very unlikely that a random point will be "extreme" along any dimension). But in a 10,000-dimensional unit hypercube (a 1 x 1 x ... x 1 cube, with ten thousand 1s), this probability is greater than 99.999999%. Most points in a high-dimensional hypercube are very close to the border.

Here is a more troublesome difference: if you pick two points randomly on a unit square, the distance between these two points will be, on average, roughly 0.52. If you pick two random points in a unit 3D cube, the average distance will be roughly 0.66. But what about two points picked randomly in a 1,000,000-dimensional hypercube? Well the average distance, believe it or not, will be about 408.25 (roughly $\sqrt{1,000,000/6}$ )! This is quite counterintuitive: how can two points be so far apart when they both lie within the same unit hypercube? This fact implies that high-dimensional datasets are at risk of being very sparse: most training instances are likely to be far away from each other. Of course, this also means that a new instance will likely be far away from any training instance, making predictions much less reliable than in lower dimensions, since they will be based on much larger extrapolations. In short, the more dimensions the training set has, the greater the risk of overfitting it.

In theory, one solution to the curse of dimensionality could be to increase the size of the training set to reach a sufficient density of training instances. Unfortunately, in practice, the number of training instances required to reach a given density grows exponentially with the number of dimensions. With just 100 features (much less than in the MNIST problem), you would need more training instances than atoms in the observable universe in order for training instances to be within 0.1 of each other on average, assuming they were spread out uniformly across all dimensions.

---

# Main Approches for Dimensionality Reduction

Let's take a look at the two main approaches to reducing dimensionality: projection & manifold learning.

## Projection

In most real-world problems, training instances are not spread out uniformly across all dimensions. Many features are almost constant, while others are highly correlated (as discussed earlier for MNIST). As a result, all training instances actually lie within (or close to) a much lower-dimensional *subspace* of the high-dimensional space. See the 3D dataset represented below.

<img src = "Images/3D Dataset in 2D Subspace.png" width = "500" style = "margin:auto"/>

Notice that all training instances lie close to a plane: this is a lower-dimensional (2D) subspace of a high-dimensional (3D) space. Now if we project every training instance perpendicularly onto this subspace (as represented by the short lines connecting the instances to the plane), we get the new 2D dataset show in the below figure. Ta-da! We have just reduced the dataset's dimensionality from 3D to 2D. Note that the axes correpsond to new features $z_1$ & $z_2$ (the coordinates of the projections on the plane).

<img src = "Images/2D Dataset After Projection.png" width = "500" style = "margin:auto"/>

However, projection is not always the best approach to dimensionality reduction. In many cases, the subspace may twist & turn, such as in the famous *Swiss roll* toy dataset represented below.

<img src = "Images/Swiss Roll Dataset.png" width = "500" style = "margin:auto"/>

Simply projecting onto a plane (e.g., by dropping $x_3$) would squash different layers of the Swiss roll together, as shown below on the left. However, what you really want is to unroll the Swiss roll to obtain the 2D dataset on the right.

<img src = "Images/Squashing vs Unrolling Swiss Roll.png" width = "500" style = "margin:auto"/>

## Manifold Learning

The Swiss roll is an example of a 2D *manifold*. Put simply, a 2D manifold is a 2D shape that can be bent & twisted in a higher-dimensional space. More generally, a *d*-dimensional manifold is a part of an *n*-dimensional space (where *d* < *n*) that locally resembles a *d*-dimensional hyperplane. In the case of the Swiss roll, *d* = 2 & *n* = 3: it locally represents a 2D plane, but it is rolled in the third dimension.

Many dimensionality reduction algroithms work by modeling the *manifold* on which the training instances lie; this is called *Manifold Learning*. It relies on the *manifold assumption*, also called the *manifold hypothesis*, which holds that most real-world high-dimensional datasets lie close to a much lower-dimensional manifold. This assumption is very often empirically observed.

Once again, think about the MNIST dataset: all handwritten digit images have some similarities. They are mde of connected lines, the borders are white, they are more or less centered, & so on. If you randomly generated images, only a ridiculously tiny fraction of them would look like handwritten digits. In other words, the degrees of freedom available to you if you try to create a digit image are dramatically lower than the degrees of freedom you would have if you were allowed to generate any image you wanted. These constraints tend to squeeze the dataset into a lower-dimensional manifold.

The manifold assumption is often accompanied by another implicit assumption: that the task at hand (e.g., classification or regression) will be simpler if expressed in the lower-dimensional space of the manifold. For example, in the top row fo the below figure, the Swiss roll is split into two classes: in the 3D space (on the left), the decision boundary would be farily complex, but in the 2D unrolled manifold space (on the right), the decision boundary is a simple straight line.

<img src = "Images/Swiss Roll Decision Boundary.png" width = "500" style = "margin:auto"/>

However, this assumption does not always hold. For example, in the bottom row, the decision boundary is located at $x_1 = 5$. This decision boundary looks very simple in the original 3D space (a vertical plane), but it looks more complex in the unrolled manifold (a collection of four independent line segments).

In short, if you reduce the dimensionality of your training set before training a model, it will usually speed up training, but it may not always lead to a better or simpler solution; it all depends on the dataset. 

Hopefully, you now have a good sense of what the curse of dimensionality is & how dimensionality reduction algorithms can fight it, especially when the manifold assumption holds. The rest of this chapter will go through some of the most popular algorithms.

---

# PCA

*Principal Component Analysis* (PCA) is by far the most popular dimensionality reduction algorithm. First, it identifies the hyperplane that lies closest to the data, & then it projects the data onto it.

## Preserving the Variance

Before you can project the training set onto a lower-dimensional hyperplane, you first need to choose the right hyperplane. For example, a simple 2D dataset is represented on the left of the below figure, along with three different axes (i.e., one-dimensional hyperplanes). On the right is the result of the projection of the dataset onto each of these axes. As you can see, the projection onto the solid line preserves the maximum variance, while the projection onto the dotted line preserves very little variance, & the projection onto the dashed line preserves an intermediate amount of variance.

<img src = "Images/Projecting Onto A Subspace.png" width = "500" style = "margin:auto"/>

It seems reasonable to select the axis that preserves the maximum amount of variance, as it will most likely lose less information than the other projections. Another way to justify this choice is that it is the axis that minimises the mean squared distance between the original dataset & its projection onto that axis. This is the idea behind *PCA*.

## Principal Components

PCA identifies the axis that accounts for the largest amount of variance in the training set. In the previous figure, it is a solid line. It also finds a second axis, orthogonal to the first one, that accounts for the largest amount of remaining variance. In this 2D example, there is no choice: it is the dotted line. If it were a higher-dimensional dataset, PCA would also find a third axis, orthogonal to both previous axes, & a fourth, a fifth, & so on -- as many axes as the number of dimensions in the dataset.

The unit vector that defines the $i^{th}$ axis is called the *$i^{th}$ principal component* (PC). In the previous figure, the $1^{st}$ PC is $c_1$, & the $2^{nd}$ PC is $c_2$. 

So how can you find the principal components of a training set? Luckily, there is a standard matrix factorization technique called *Singular Value Decomposition* (SVD) that can decompose the training set matrix $X$ into the matrix multiplication of three matrices $U \sum V^T$, where $V$ contains all the principal components that we are looking for, as shown in the below equation.

$$V = (c_1, c_2, ..., c_n)$$

The following code uses numpy's `svd()` function to obtain all the principal components of the training set, then extracts the first two PCs:

In [1]:
import numpy as np

np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

## Projecting Down to *d* Dimensions

Once you have identified all the principal components, you can reduce the dimensionality of the dataset down to *d* dimensions by projecting it onto the hyperplane defined by the first *d* principal components. Selecting this hyperplane ensures that the projection will preserve as much variance as possible. For example, when we projected our 3D dataset down to the 2D plane, defined by the first two principal components, preserving a large part of the dataset's variance. As a result, the 2D projection looks very much like the original 3D dataset.

To project the training set onto the hyperplane, you can simply compute the matrix multiplication of the training set matrix $X$ by the matrix $W_d$, defined as the matrix containing the first $d$ principal components (i.e., the matrix composed of the first *d* columns of $V$), as shown in the below equation.

$$X_{d-proj} - XW_d$$

The following python code projects the training set onto the plane defined by the first two principal components:

In [2]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

There you have it. You now know how to reduce the dimensionality of any dataset down to any number of dimensions, while preserving as much variance as possible.

## Using Scikit-Learn

Scikit-learn's PCA class implements PCA using SVD decomposition just like we did before. The following code applies PCA to reduce the dimensionality of the dataset down to two dimensions (note that it automatically takes care of centering the data):

In [3]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

After fitting the `PCA` transformer to the dataset, you can access the principal components using the `components_` variable (not that it contains the PCs as horizontal vectors, so, for example, the first principal component is equal to `pca.components_.T[:, 0]`).

## Explained Variance Ratio

Another very useful piece of information is the *explained variance ration* of each principal component, available via the `explained_variance_ratio_` variable. It indicates the proportion of the dataset's variance that lies along the axis of each principal component. For example, let's look at the explained variance ratios of the first two components of the 3D dataset that we projected onto a 2D subspace.

In [4]:
pca.explained_variance_ratio_

array([0.84248607, 0.14631839])

This tells us that 84.2% of the dataset's variance lies along the first axis, & 14.6% lies along the second axis. This leaves less than 1.2% for the third axis, so it is reasonable to assume that it probably carries little information.

## Choosing the Right Number of Dimensions

Instead of arbitrarily choosing the number of dimensions to reduce it down to, it is generally preferable to choose the number of dimensions that add up to a sufficiently large portion of the variance (e.g., 95%). Unless, of course, you are reducing dimensionality for data visualisation -- in that case, you will generally want to reduce the dimensionality down to 2 or 3. The following code computes PCA without reducing dimensionality, then computes the minimum of dimensions required to preserve 95% of the training set's variance:

In [5]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml('mnist_784', version = 1, as_frame = False, parser = "auto")
mnist.target = mnist.target.astype(np.uint8)
X = mnist["data"]
y = mnist["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

You could then set `n_components = d` & run PCA again. However, there is a much better option: instead of specifying the number of principal components you want to preserve, you can set `n_componets` to be a float between `0.0` & `1.0`, indicating the ratio of variance you wish to preserve.

In [6]:
pca = PCA(n_components = 0.95)
X_reduced = pca.fit_transform(X_train)

Yet another option is to plot the exaplined variance as a function of the number of dimensions (simply plot `cumsum`). There will usually be an elbow in the curve, where the explained variance stops growing fast. You can think of this as the intrinsic dimensionality of the dataset. In this case, you can see that reducing the dimensionality down to about 100 dimensions wouldn't lose too much explained variance.

<img src = "Images/Explained Variance as Function of Number of Dimensions.png" width = "500" style = "margin:auto"/>

## PCA for Compression

Obviously after dimensionality reduction, the training set takes up much less space. For example, try appling PCA to the MNIST dataset while preserving 95% of its variance. You should find that each instance will have just over 150 features, instead of the original 784 features. So while most of the variance is preserved, the dataset is now less than 20% of its original size! This is a reasonable compression ratio, & you can see how this can speed up a classification algorithm (such as an SVM classifier) tremendously.

It is almost possible to decompress the reduced dataset back to 784 dimensions by applying the inverse transformation of the PCA projection. Of course this won't give you back to the original data, since the projection lost a bit of information (within the 5% variance that was dropped), but it will likely be quite close to the original data. The mean squared distance between the original data & the reconstructed data (compressed & then decompressed) is called the *reconstruction error*. For example, the following code compresses the MNIST dataset down to 154 dimensions, then uses the `inverse_transform()` method to decompress it back to 784 dimensions. The below figure shows a few digits from the original training set (on the left) & the corresponding digits after compression & decompression. You can see that there is a slight image quality loss, but the digits are still mostly intact.

In [7]:
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

<img src = "Images/MNIST Compression Preservation.png" width = "500" style = "margin:auto"/>

The equation of the inverse transformation is shown in the equation below.

$$X_{recovered} = X_{d-proj}W_d^T$$

## Randomised PCA

If you set the `svd_solver` hyperparameter to `"randomised"`, scikit-learn uses a stochastic algorithm called *Randomised PCA* that quickly finds an approximation of the first *d* principal components. Its computational complexity is $O(m * d^2) + O(d^3)$, instead of $O(m * n^2) + O(n^3)$ for the full SVD approach, so it is dramatically faster than full SVD when *d* is much smaller than *n*:

In [8]:
rnd_pca = PCA(n_components = 154, svd_solver = "randomized")
X_reduced = rnd_pca.fit_transform(X_train)

By default, `svd_solver` is actually set to `"auto"`: scikit-learn automatically uses the randomised PCA algorithm if *m* or *n* is greated than 500 & *d* is less than 80% of *m* or *n*, or else it uses the full SVD approach. If you want to force scikit-learn to use full SVD, you can set the `svd_solver` hyperparameter to `"full"`.

## Incremental PCA

One problem with preceding implementations of PCA is that they require the whole training set to fit in memory in order for the algorithm to run. Fortunately, *Incremental PCA* (IPCA) algorithms have been developed: you can split the training set into mini-batches & feed an IPCA algorithm one mini-batch at a time. This is useful for large training sets, & also to apply PCA online (i.e., on the fly, as new instances arrive). The following code splits the MNIST dataset into 100 mini-batches (using numpy's `array_split()` function) & feeds them to scikit-learn's `IncrementalPCA` class to reduce the dimensionality of the MNIST dataset down to 154 dimensions (just like before). Not that you must call the `partial_fit()` method with each mini-batch rather than the `fit()` method with the whole training set.

In [9]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components = 154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

Alternatively, you can use numpy's `memmap` class, which allows you to manipulate a large array stored in a binary file on disk as if it were entirely in memory; the class loads only the dataset it needs in memory, when it needs it. Since the `IncrementalPCA` class uses only a small part of the array at any given time, the memory usage remains under control. This makes it possible to call the usual `fit()` method, as you can see in the following code:

In [10]:
filename = "my_mnist.data"
m, n = X_train.shape

X_mm = np.memmap(filename, dtype = 'float32', mode = 'write', shape = (m, n))
X_mm[:] = X_train

del X_mm

X_mm = np.memmap(filename, dtype = "float32", mode = "readonly", shape = (m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components = 154, batch_size = batch_size)
inc_pca.fit(X_mm)

IncrementalPCA(batch_size=525, n_components=154)

---

# Kernel PCA

In previous lessons, we discussed the kernel trick, a mathematical technique that implicitly maps instances into a very high-dimensional space (called the *feature space*), enabling nonlinear classification & regression with support vector machines. Recall that a linear decision boundary in the high-dimensional feature space corresponds to a complex nonlinear decision boundary in the *original space*. 

It turns out that the same trick can be applied to PCA, making it possible to perform complex nonlinear projections for dimensionality reduction. This is called *Kernel PCA* (kPCA). It is often good at preserving clusters of instances after projection, or sometimes even unrolling datasets that lie close to a twisted manifold.

For example, the following code uses scikit-learn's `KernelPCA` class to perform kPCA with an RBF kernel.

In [11]:
from sklearn.decomposition import KernelPCA

# Takes way too long
#rbf_pca = KernelPCA(n_components = 2, kernel = "rbf", gamma = 0.04)
#X_reduced = rbf_pca.fit_transform(X)

The below figure shows the Swiss roll, reduced to two dimensions using a linear kernel (equivalent to simply using the `PCA` class), an RBF kernel, & a sigmoid kernel (logistic).

## Selecting a Kernel & Tuning Hyperparameters

As kPCA is an unsupervised learning algorithm, there is no obvious performance measure to help you select the best kernel & hyperparameter values. However, dimensioality reduction is often a preparation step for a supervised learning task (e.g., classification), so you can simply use grid search the kernel & hyperparameters that lead to the best performance on that task. For example, the following code creates a two-step pipeline, first reducing dimensionality to two dimensions using kPCA, then applying logistic regression for classification. Then it uses `GridSearchCV` to find the best kernel & gamma value for kPCA in order to get the best classification accuracy at the end of the pipeline.

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Takes way too long
#classifier = Pipeline([("kpca", KernelPCA(n_components = 2)),
#                       ("log_reg", LogisticRegression())])
#param_search_space = [{"kpca__gamma": np.linspace(0.03, 0.05, 10),
#                       "kpca__kernel":["rbf", "sigmoid"]}]
#grid_search = GridSearchCV(classifier, param_search_space, cv = 3)
#grid_search.fit(X, y)
#grid_Search.best_params_

Another approach, this time entirely unsupervised, is to select a kernel & hyperparameters that yield the lowest reconstruction error. However, reconstruction is not as easy as with linear PCA. Here's why. The below figure shows the original Swiss roll 3D dataset (top left), & the resulting 2D dataset after kPCA is applied using an RBF kernel (top right). Thanks to the kernel trick, this is mathematically equivalent to mapping the training set to an infinite-dimensional feature space (bottom right) using the *feature map* $\varphi$, then projecting the transformed training set down to 2D using linear PCA. Notice that if we could invert the linear PCA step for a given instance in the reduced space, the reconstructed point would lie in the feature space, not in the original space (e.g., like the one represented by an x in the diagram). Since the feature space is infinite-dimensional, we cannot compute the reconstructed point, & therefore we cannot compute the true reconstruction error. Fortunately, it is possible to find a point in the original space that would map close to the reconstructed point. This is called the reconstruction *pre-image*. Once you have this pre-image, you can measure its squared distance to the original instance. You can then select the kernel & hyperparameters that minimise this reconstruction pre-image error.

<img src = "Images/Kernel PCA & Reconstruction Pre-Image Error.png" width = "450" style = "margin:auto"/>

You may be wondering how to perform this reconstruction. One solution is to train a supervised regression model, with the projected instances as the training set & the original instances as the targets. Scikit-lelarn will do this automatically if you set `fit_inverse_transform = True`, as shown in the folowing code:

In [13]:
# Takes way too long
#rbf_pca = KernelPCA(n_components = 2, kernel = "rbf", gamma = 0.0433,
#                    fit_inverse_transform = True)
#X_reduced = rbf_pca.fit_transform(X)
#X_preimage = rbf_pca.inverse_transform(X_reduced)

You can then compute the reconstruction pre-image error:

In [14]:
from sklearn.metrics import mean_squared_error

# Takes way too long
#mean_squared_error(X, X_preimage)

Now you can use grid search with cross-validation to find the kernel & hyperparameters that minimise this pre-image reconstruction error.

---

# LLE

*Local Linear Embedding* (LLE) is another very powerful *nonlinear dimensionality reduction* (NLDR) technique. It is a manifold learning technique that does not rely on projections like the previous algorithms. In a nuteshel, LLE works by first measuring how each training instance linearly relates to its closest neighbors, & then looking for a low-dimensional representation of the training set where these local relationships are best preserved. This makes it particularly good at unrolling twisted manifolds, especially when there is not too much noise.

For example, the following code uses scikit-learn's `LocallyLinearEmbedding` class to unroll the Swiss roll. The resulting 2D dataset is shown.

<img src = "Images/Unroll Swiss Roll with LLE.png" width = "450" style = "margin:auto"/>

As you can see, the Swiss roll is completely unrolled & the distances between instances are locally well preserved. However, distances are not preserved on a larger scale: the left part of the unrolled swiss roll is stretched, while the right part is squeezed. Nevertheless, LLE did a pretty good job at modeling the manifold.

In [15]:
from sklearn.manifold import LocallyLinearEmbedding

# Takes too long
#lle = LocallyLinearEmbedding(n_components = 2, n_neighbors = 10)
#X_reduced = lle.fit_transform(X)

Here's how LLE works: first, for each training instance $x^{(i)}$, the algorithm identifies its *k* closest neighbors (in the preceding code *k* = 10), then tries to reconstruct $x^{(i)}$ as a linear function of these neighbors. More specifically, it finds the weights $w_{i, j}$ such that the squared distance between $x^{(i)}$ & $\sum^{m}_{j = 1}w_{i, j}x^{(j)}$ is as small as possible, assuming $w_{i, j} = 0$ if $x^{(j)}$ is not one of the *k* closest neighbors of $x^{(i)}$. Thus the first step of LLE is the constrained optimisation problem described in the below eqution, where $W$ is the weight matrix containing all the weights $w_{i, j}$. The second constraint simply normalises the weights for each training instance $x^{(i)}$.

$$\begin{split}
\hat{W} = \underset{w}{argmin} \sum^{m}_{i = 1} (x^{(i)} - \sum^{m}_{j = 1} w_{i, j}x^{(j)})^2 \\
subject\ to\ \Biggl\{\begin{split}
w_{i, j} = 0 \quad if\ x^{(j)}\ is\ not\ one\ of\ the\ k\ closest\ neighbors\ of\ x^{(i)}\\
\sum^{m}_{j = 1} m_{i, j} = 1 \quad for\ i = 1, 2, ..., m
\end{split}
\end{split}$$

After this step, the weight matrix $\hat{W}$ (containing the weights $\hat{w}_{i, j}$) encodes the local linear relationships between the training instances. Now the second step is to map the training instances into a *d*-dimensional space (where *d* < *n*) while preserving these local relationships as much as possible. If $z^{(i)}$ is the image of $x^{(i)}$ in this *d*-dimensional space, then we want the squared distance between $z^{(i)}$ & $\sum^{m}_{j = 1}\hat{w}_{i, j}z^{(j)}$ to be as small as possible. This idea leads to the unconstrained optimisation problem described in the below equation. It looks similar to the first step, but instead of keeping the instances fixed & finding the optimal position of the instance's images in the low-dimensional space. Note that $Z$ is the matrix containing all $z^{(i)}$.

$$\hat{Z} = \underset{Z}{argmin} \sum^{m}_{i = 1} (z^{(i)} - \sum^{m}_{j = 1} \hat{w}_{i, j}z^{(j)})^2$$

Scikit-learn's LLE implementation has the following computational complexity: $O(m\ log(m)n\ log(k))$ for finding the *k* nearest neighbors, $O(mnk^3)$ for optimising the weights, & $O(dm^2)$ for constructing the low-dimensional representations. Unfortunately, the $m^2$ in the last term makes this algorithm scale poorly to very large datasets.

---

# Other Dimensionality Reduction Techniques

There are many other dimensionality reduction techniques, several of which are available in scikit-learn. Here are some of the most popular:

* *Multidimensional Scaling* (MDS) reduces dimensionality while trying to preserve the distances between the instances.
* *Isomap* creates a graph by connecting each instance to its nearest neighbors, then reduces dimensionality while trying to preserve the *geodesic distances* between the instances.
* *t_Distributed Stochastic Neighbors Embedding* (t-SNE) reduces dimensionality while trying to keep similar instances close & dissimilar instances apart. It is mostly used for visualisation, in particular to visualise clusters of instances in high-dimensional space (e.g., to visualise the MNIST images in 2D).
* *Linear Discriminant Analysis* (LDA) is actually a classification algorithm, but during training, it learns the most discriminative axes between the classes, & these axes can then be used to define a hyperplane onto which to project the data. The benefit is that the projection will keep classes as far apart as possible, so LDA is a good technique to reduce dimensionality before running another classification algorithm such as an SVM classifier.

<img src = "Images/Various Dimensionality Reduction Techniques.png" width = "600" style = "margin:auto"/>